## Create Table with TIMS data 

##### June 17, 2019

In this notebook, we are prototyping code to 
1. Create a PSQL table with the TIMS data. 
2. Update the PSQL table with all new csv files 

In [36]:
import pandas as pd
import numpy as np
import sys, os
import boto3
import pyproj
import datetime

basepath = os.getcwd() #path of current script

List TIMS data from S3

In [37]:
s3_session = boto3.Session(profile_name='dssg')
bucket_name = 'air-pollution-uk'

s3_resource = s3_session.resource('s3')

my_bucket = s3_resource.Bucket(bucket_name)


In [38]:
objects = my_bucket.objects.filter(Prefix="raw/tims_data/")


#### Check number of videos pulled per month

Speculation: All the videos are pulled for the 1st day of each month. 

In [39]:
date_current = datetime.date(2019, 1, 1)
date_end = datetime.date(2019, 1, 4)
day = datetime.timedelta(days=1)

while date_current <= date_end:
    counter = 0
    for obj in objects:
        filepath = obj.key
        filename = filepath.split("/")[-1]
        prefix = "detdata" + date_current.strftime('%m%d%Y')
        if filename.startswith(prefix):
            counter+=1
    print(date_current.strftime(ls
                                '%m%d%Y'), counter)
    
    date_current += day

01012019 96
01022019 96
01032019 96
01042019 90


In [40]:
#check a csv 
s3_client = s3_session.client('s3')
csv_obj = s3_client.get_object(Bucket=bucket_name, Key='raw/tims_data/detdata01042019-213027.csv')
initial_df = pd.read_csv(csv_obj['Body']) # 'Body' is a key word
initial_df.head()

TIMESTAMP    NODE    EASTING   NORTHING  FLOW_ACTUAL_15M  \
0  2019-04-01T20:21:00Z  10/114  528894.67  177070.33              211   
1  2019-04-01T20:21:00Z  10/115  528385.87  173071.87              382   
2  2019-04-01T20:21:00Z  10/116  528399.50  173073.96              407   
3  2019-04-01T20:21:00Z  10/117  528294.39  172880.22              226   
4  2019-04-01T20:21:00Z  10/118  528306.71  172882.36              444   

  SAT_BANDINGS  DETECTOR_NO  TOTAL_DETECTOR_NO  DETECTOR_RATE  
0        0-79%            4                  4            1.0  
1        0-79%            1                  1            1.0  
2        0-79%            1                  1            1.0  
3        0-79%            2                  2            1.0  
4        0-79%            1                  1            1.0